In [1]:
! pip uninstall torch torchvision -y


In [2]:
! pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
! pip install -U transformers
! pip install -U simpletransformers  

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu101/torch-1.6.0%2Bcu101-cp38-cp38-win_amd64.whl (1021.6 MB)
  Using cached https://download.pytorch.org/whl/cu101/torchvision-0.7.0%2Bcu101-cp38-cp38-win_amd64.whl (1.1 MB)
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=b3e22e344fcf4240c0bfefa14075fdec6519d9cd086f471fb1fd0c1eb2af3240
  Stored in directory: c:\users\valon_januzaj\appdata\local\pip\cache\wheels\8e\70\28\3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future
  Using cached transformers-4.8.2-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp38-cp38-win_amd64.whl (2.0 MB)
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Using cached simpletransformers-0.61.9-py3-none-any.whl (220 kB)
  Using cached tensorboardX-2.4-py2.py3-none-any.whl (124 kB)
  Using cached s

In [4]:
import json

data_file = 'arxiv-metadata-oai-snapshot.json'

# Helper function to load the dataset
def get_metadata():
    with open(data_file, 'r') as f:
        for line in f:
            yield line

In [5]:
# let's see the first row of the data
metadata = get_metadata()
for paper in metadata:
    paper_dict = json.loads(paper)
    print('Title: {}\n\nAbstract: {}\nRef: {}'.format(paper_dict.get('title'), paper_dict.get('abstract'), paper_dict.get('journal-ref')))
#     print(paper)
    break

Title: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies

Abstract:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
more detailed tests with CDF and DO data. Predictions are shown for
distributions of diphoton pairs produced at the energy of the Large Hadron
Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs
boson are contrasted with those produced from QCD processes at the LHC, showing
tha

In [6]:
titles = []
abstracts = []
years = []
metadata = get_metadata()
for paper in metadata:
    paper_dict = json.loads(paper)
    ref = paper_dict.get('journal-ref')
    try:
        year = int(ref[-4:]) 
        if 2020 < year < 2022:
            if len(titles) == 500:
                break
            years.append(year)
            titles.append(paper_dict.get('title'))
            abstracts.append(paper_dict.get('abstract'))
    except:
        pass 

len(titles), len(abstracts), len(years)

(500, 500, 500)

In [8]:
import pandas as pd
papers = pd.DataFrame({
    'title': titles,
    'abstract': abstracts,
    'year': years
})
papers.head()

,title,abstract,year
0,On Path diagrams and Stirling permutations,A permutation can be locally classified acco...,2021
1,Onset of inertial magnetoconvection in rotatin...,The onset of convection in the form of magne...,2021
2,A model of compact polymers on a family of thr...,We study Hamiltonian walks (HWs) on the fami...,2021
3,A systematic approach to the Planck LFI end-to...,The Level 1 of the Planck LFI Data Processin...,2021
4,Gravitational waves from cosmological phase tr...,I discuss the generation of a stochastic bac...,2021


In [9]:
# Adding <input_text> and <target_text> columns
papers = papers[['title','abstract']]
papers.columns = ['input_text', 'target_text']

# Adding <prefix> columns
papers['prefix'] = "summarize"

# splitting the data into training and test dataset
eval_df = papers.sample(frac=0.2, random_state=101)
train_df = papers.drop(eval_df.index)

train_df.shape, eval_df.shape

((400, 3), (100, 3))

In [10]:
import logging

import pandas as pd
from simpletransformers.t5 import T5Model

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# T5 Training parameters
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "train_batch_size": 16,
    "num_train_epochs": 4,
}

# Create T5 Model
model = T5Model(model_name="t5-small",model_type="t5", args=model_args, use_cuda=False)

# Train T5 Model on new task
model.train_model(train_df)

# Evaluate T5 Model on new task
results = model.eval_model(eval_df)

print(results)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-small_cached_512400
INFO:simpletransformers.t5.t5_model: Training started



Using Adafactor for T5


C:\Users\valon_januzaj\anaconda3\envs\python-model\lib\site-packages\torch\optim\lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.t5.t5_model: Training of t5-small model complete. Saved to outputs/.
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-small_cached_512100


INFO:simpletransformers.t5.t5_model:{'eval_loss': 3.8267076932466946}



{'eval_loss': 3.8267076932466946}
